In [79]:
#import libraries
import altair as alt
import pandas as pd
import numpy as np
from scipy.stats import theilslopes, linregress
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
alt.data_transformers.enable('vegafusion')
import ipywidgets as widgets
from IPython.display import display, clear_output


In [80]:

# dictionary containing aquifer data
aquifers = {
    'Ogallala': {
        'color': 'Reds',
        'path': '/Users/finleydavis/Desktop/Cardenas Research/Raw Data/Parsed Aquifers/Date Sorted/corrected/Ogallala.csv'
    },
    'Edwards (Balcones Fault Zone)': {
        'color': 'Oranges',
        'path': '/Users/finleydavis/Desktop/Cardenas Research/Raw Data/Parsed Aquifers/Date Sorted/corrected/Edwards (Balcones Fault Zone) Aquifer.csv'
    },
    'Edwards-Trinity Plateau': {
        'color': 'YlOrBr',
        'path': '/Users/finleydavis/Desktop/Cardenas Research/Raw Data/Parsed Aquifers/Date Sorted/corrected/Edwards-Trinity Plateau.csv'
    },
    'Carrizo-Wilcox': {
        'color': 'Greens',
        'path': '/Users/finleydavis/Desktop/Cardenas Research/Raw Data/Parsed Aquifers/Date Sorted/corrected/Carrizo-Wilcox.csv'
    },
    'Gulf Coast': {
        'color': 'Blues',
        'path': '/Users/finleydavis/Desktop/Cardenas Research/Raw Data/Parsed Aquifers/Date Sorted/corrected/Gulf Coast.csv'
    },
    'Pecos Valley': {
        'color': 'indigo',
        'path': '/Users/finleydavis/Desktop/Cardenas Research/Raw Data/Parsed Aquifers/Date Sorted/corrected/Pecos Valley.csv'
    },
    'Seymour': {
        'color': 'Purples',
        'path': '/Users/finleydavis/Desktop/Cardenas Research/Raw Data/Parsed Aquifers/Date Sorted/corrected/Seymour.csv'
    },
    'Trinity': {
        'color': 'magenta',
        'path': '/Users/finleydavis/Desktop/Cardenas Research/Raw Data/Parsed Aquifers/Date Sorted/corrected/Trinity.csv'
    },
    'Hueco-Mesilla Basin': {
        'color': 'Pinks',
        'path': '/Users/finleydavis/Desktop/Cardenas Research/Raw Data/Parsed Aquifers/Date Sorted/corrected/Hueco-Mesilla Basin.csv'
    }
}

In [81]:
#allows large datasets for altair
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('vegafusion')

In [82]:
#ln mean computation
def lognormal_mean(x):
    if len(x) == 1:
        return x.iloc[0]
    log_x = np.log(x)
    return np.exp(log_x.mean() + 0.5 * log_x.std() ** 2)

In [ ]:
#theil-sen trend and bootstrap confidence interval; Not using this currently
def compute_trend_and_ci(years, depths, n_boot=1000, ci_percent=90):
    """
    Compute Theil-Sen slope, intercept, and confidence interval via bootstrapping
    """
    slope, intercept, *_ = theilslopes(depths, years)
    trend = intercept + slope * years
    
    # bootstrap CI
    boot_trends = np.zeros((n_boot, len(years)))
    for i in range(n_boot):
        idx = np.random.choice(len(years), size=len(years), replace=True)
        boot_slope, boot_intercept, *_ = theilslopes(depths[idx], years[idx])
        boot_trends[i] = boot_intercept + boot_slope * years
    lower_ci = np.percentile(boot_trends, (100-ci_percent)/2, axis=0)
    upper_ci = np.percentile(boot_trends, 100-(100-ci_percent)/2, axis=0)
    
    return trend, lower_ci, upper_ci, slope, intercept

In [ ]:
#plot
def analyze_aquifer_data_html(file_path, aquifer_name, aquifer_color, start_date=1920, end_date=2023):

    dtype = {'ID':'int32','Lat':'float32','Long':'float32','County':'category','Date':'int16','Depth':'float32'}
    df = pd.read_csv(file_path, dtype=dtype)
    df = df.dropna(axis=1, how='all')
    df.columns = ['ID','Lat','Long','County','Date','Depth']

    df = df[(df.Date.between(start_date,end_date)) & (df.Depth>0)].copy()
    df['Date'] = pd.to_datetime(df['Date'], format='%Y')

    def lognormal_mean(x):
        if len(x)==1: return x.iloc[0]
        log_x = np.log(x)
        return np.exp(log_x.mean() + 0.5*log_x.std()**2)
    
    annual_means = df.groupby('Date')['Depth'].agg(lognormal_mean).reset_index()

    #Calc theil-sen trend
    years_num = np.array([d.year for d in annual_means['Date']])
    values = annual_means['Depth'].values
    slope, intercept, *_ = theilslopes(values, years_num)
    trend_line = intercept + slope * years_num
    annual_means['Trend'] = trend_line

    #CI, not in use rn
    annual_means['Lower_CI'] = trend_line - 0.1*np.mean(values)
    annual_means['Upper_CI'] = trend_line + 0.1*np.mean(values)

    #altair plot
    scatter = alt.Chart(df).mark_circle(size=20, opacity=0.2).encode(
        x='Date:T', y='Depth:Q', tooltip=['ID','County','Depth']
    )

    mean_line = alt.Chart(annual_means).mark_line(color='black').encode(
        x='Date:T', y='Depth:Q'
    )

    trend_line_chart = alt.Chart(annual_means).mark_line(color='red', strokeDash=[5,5]).encode(
        x='Date:T', y='Trend:Q'
    )

    ci_band = alt.Chart(annual_means).mark_area(opacity=0.3, color=aquifer_color).encode(
        x='Date:T', y='Lower_CI:Q', y2='Upper_CI:Q'
    )

    chart = alt.layer(scatter, mean_line, trend_line_chart).properties(
        width=800, height=400,
        title=f'{aquifer_name} Aquifer Depth Analysis ({start_date}-{end_date})'
    ).interactive()

    return chart


In [116]:
chart = analyze_aquifer_data_html(aquifers['Ogallala']['path'], 'Ogallala', 'blue')

In [117]:
chart.save('/Users/finleydavis/Desktop/Cardenas Research/Python/_repo/TWD/html/plots/ogallala_aquifer_interactive.html')